# Web scraping Elipso

In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Zlatko\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [9]:
url ="https://www.pevex.hr"
driver.get(url)

In [10]:
def get_url(search_term):
    """Generate a url from search term"""
    template="https://pevex.hr/catalogsearch/result/?q={}"
    search_term=search_term.replace(" ","+")
    return template.format(search_term)

In [12]:
url=get_url("led tv")
print(url)

https://pevex.hr/catalogsearch/result/?q=led+tv


In [13]:
driver.get(url)

### Extract the collection

In [14]:
soup= BeautifulSoup(driver.page_source,"html.parser")

In [26]:
results=soup.find_all("li","item product product-item")

In [39]:
len(results)

16

### Prototype the record

In [40]:
item=results[0]

In [47]:
atag=item.strong.a

In [49]:
description=atag.text.strip()

'TV LED HITACHI 32HAE2250'

In [51]:
url=atag.get("href")

In [57]:
price=item.find("span","price").text.replace("\xa0","")
price

'1.699,00kn'

### Generalize the pattern

In [58]:
def extract_record(item):
    """ Extract and return data from a single record"""
    #Description and url
    atag=item.strong.a
    description=atag.text.strip()
    url=url=atag.get("href")
    
    #price
    price=item.find("span","price").text.replace("\xa0","")
    
    
    results=(description,price,url)
    
    return results

In [62]:
records=[]
results=soup.find_all("li","item product product-item")
for item in results:
    records.append(extract_record(item))

In [63]:
records[0]

('TV LED HITACHI 32HAE2250',
 '1.699,00kn',
 'https://pevex.hr/tv-led-hitachi-32hae2250-hd-ready-dvb-t2-s2-android-340863.html')

In [65]:
for row in records:
    print(row[1])

1.699,00kn
3.899,00kn
3.499,00kn
1.299,00kn
2.799,00kn
2.949,00kn
3.699,00kn
4.509,00kn
2.589,00kn
3.139,00kn
3.999,00kn
3.599,00kn
5.499,00kn
7.939,00kn
4.699,00kn
11.499,00kn


### Going to next page

In [66]:
def get_url(search_term):
    """Generate a url from search term"""
    template="https://pevex.hr/catalogsearch/result/?q={}"
    search_term=search_term.replace(" ","+")
    
    
    #add query to url
    url=template.format(search_term)
    #add page query placeholder
    url+= "index/?p={}"
    return url

### Putting all together

In [80]:
import csv
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver



def get_url(search_term):
    """Generate a url from search term"""
    template="https://pevex.hr/catalogsearch/result/?q={}"
    search_term=search_term.replace(" ","+")
    
    
    #add query to url
    url=template.format(search_term)
    #add page query placeholder
    url="https://pevex.hr/catalogsearch/result/index/?p={}&q=led+tv"
    return url

def extract_record(item):
    """ Extract and return data from a single record"""
    #Description and url
    atag=item.strong.a
    description=atag.text.strip()
    url=url=atag.get("href")
    
    #price
    price=item.find("span","price").text.replace("\xa0","")
    
    
    results=(description,price,url)
    
    return results


def main(search_term):
    """ Run main program routime"""
    #startup the webdriver
    driver=driver = webdriver.Chrome(ChromeDriverManager().install())
    
    record=[]
    url=get_url(search_term)
    #we have 10 page of search in our example for "led tv"
    for page in range (1,11):
        driver.get(url.format(page))
        soup=BeautifulSoup(driver.page_source,"html.parser")
        results=soup.find_all("li","item product product-item")
        
        for item in results:
            record=extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save data to csv file
    with open("elipso_results.csv","w",encoding="utf-8") as f:
        writer=csv.writer(f)
        writer.writerow(["Description","Price","Url"])
        writer.writerows(records)

In [79]:
main("led tv")

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\Zlatko\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
